In [12]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error, mean_squared_error
import os


# Industrial Sector

In [13]:
# Read the Excel file
# Data_Status: Indicates the status of the data. The value "2020F" suggests that it is a forecast for the year 2020.
# State: Represents the state for which the data is recorded (in this case, "CA" for California).
# MSN: Stands for "Monthly State Names" and refers to the specific energy metric or variable being measured. Examples include ARICD, ARICV, ARTCD, ARTCV, ARTXD, WWTXV, WXICD, WXICV, ZWCDP, ZWHDP.
df = pd.read_excel('Datasets/use_tot_sector.xlsx' , sheet_name='Industrial Sector')


In [14]:
df.head()

,State,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AK,16798,21748,25143,25950,25841,23107,33820,35548,36499,...,331425,339187,334290,330437,331233,329936,333699,334078,335987,363454
1,AL,508500,473290,497504,505064,558585,614582,642039,681691,720272,...,804136,835603,850483,861148,823667,813733,827644,845914,819961,773188
2,AR,204729,199953,216451,230995,255283,255234,257650,277382,285240,...,421863,405171,412224,412792,377648,381146,397376,411175,400559,377372
3,AZ,50359,81565,97937,105519,100297,119098,131975,119078,128483,...,224630,219917,213684,236623,236019,240833,231833,229212,226423,224452
4,CA,1193589,1209024,1214818,1325160,1454644,1475421,1558212,1610012,1697912,...,1783641,1746357,1884236,1859708,1849423,1851514,1828150,1852054,1806508,1701239


# Data statistics

In [15]:
df.head()

,State,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AK,16798,21748,25143,25950,25841,23107,33820,35548,36499,...,331425,339187,334290,330437,331233,329936,333699,334078,335987,363454
1,AL,508500,473290,497504,505064,558585,614582,642039,681691,720272,...,804136,835603,850483,861148,823667,813733,827644,845914,819961,773188
2,AR,204729,199953,216451,230995,255283,255234,257650,277382,285240,...,421863,405171,412224,412792,377648,381146,397376,411175,400559,377372
3,AZ,50359,81565,97937,105519,100297,119098,131975,119078,128483,...,224630,219917,213684,236623,236019,240833,231833,229212,226423,224452
4,CA,1193589,1209024,1214818,1325160,1454644,1475421,1558212,1610012,1697912,...,1783641,1746357,1884236,1859708,1849423,1851514,1828150,1852054,1806508,1701239


In [16]:
df.describe()

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
count,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,...,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01
mean,8.000753e+05,8.044438e+05,8.364175e+05,8.741314e+05,9.261560e+05,9.641744e+05,1.015282e+06,1.024852e+06,1.072641e+06,1.119337e+06,...,1.191517e+06,1.194068e+06,1.215542e+06,1.222737e+06,1.209438e+06,1.208458e+06,1.226697e+06,1.262752e+06,1.259703e+06,1.202560e+06
std,2.879433e+06,2.893628e+06,3.007903e+06,3.144656e+06,3.329957e+06,3.466424e+06,3.650322e+06,3.687599e+06,3.859821e+06,4.026843e+06,...,4.306187e+06,4.317862e+06,4.400481e+06,4.424566e+06,4.382080e+06,4.383491e+06,4.451797e+06,4.591711e+06,4.586021e+06,4.382147e+06
min,1.395800e+04,1.457200e+04,1.500600e+04,1.387400e+04,1.315700e+04,1.671400e+04,1.941600e+04,1.971400e+04,2.125000e+04,2.260300e+04,...,6.817000e+03,7.015000e+03,7.152000e+03,7.260000e+03,6.959000e+03,5.821000e+03,5.659000e+03,5.582000e+03,4.895000e+03,4.841000e+03
25%,8.576025e+04,9.210325e+04,9.776400e+04,9.853425e+04,1.012090e+05,1.117292e+05,1.214362e+05,1.162295e+05,1.277455e+05,1.415455e+05,...,2.103712e+05,2.102555e+05,2.048248e+05,1.818842e+05,1.720935e+05,1.715650e+05,1.720048e+05,1.730418e+05,1.717345e+05,1.684770e+05
50%,2.403565e+05,2.466050e+05,2.632880e+05,2.620795e+05,2.765970e+05,3.058610e+05,3.311005e+05,3.026695e+05,3.279200e+05,3.374545e+05,...,3.674945e+05,3.772125e+05,3.848425e+05,3.855620e+05,3.804010e+05,3.867255e+05,3.906125e+05,3.946290e+05,3.885570e+05,3.790090e+05
75%,4.835130e+05,4.740025e+05,4.794185e+05,5.046800e+05,5.186458e+05,5.335820e+05,5.499260e+05,5.722648e+05,5.977088e+05,6.313035e+05,...,6.691142e+05,6.626002e+05,6.645230e+05,6.658832e+05,6.212650e+05,6.197728e+05,6.294400e+05,6.428698e+05,6.350805e+05,5.875778e+05
max,2.079914e+07,2.091160e+07,2.174410e+07,2.272372e+07,2.407484e+07,2.505931e+07,2.638487e+07,2.663848e+07,2.788000e+07,2.908470e+07,...,3.098498e+07,3.104775e+07,3.159538e+07,3.178039e+07,3.143651e+07,3.141034e+07,3.187981e+07,3.281873e+07,3.274172e+07,3.126008e+07


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 62 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   State   52 non-null     object
 1   1960    52 non-null     int64 
 2   1961    52 non-null     int64 
 3   1962    52 non-null     int64 
 4   1963    52 non-null     int64 
 5   1964    52 non-null     int64 
 6   1965    52 non-null     int64 
 7   1966    52 non-null     int64 
 8   1967    52 non-null     int64 
 9   1968    52 non-null     int64 
 10  1969    52 non-null     int64 
 11  1970    52 non-null     int64 
 12  1971    52 non-null     int64 
 13  1972    52 non-null     int64 
 14  1973    52 non-null     int64 
 15  1974    52 non-null     int64 
 16  1975    52 non-null     int64 
 17  1976    52 non-null     int64 
 18  1977    52 non-null     int64 
 19  1978    52 non-null     int64 
 20  1979    52 non-null     int64 
 21  1980    52 non-null     int64 
 22  1981    52 non-null     int6

In [18]:
df.head()

,State,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AK,16798,21748,25143,25950,25841,23107,33820,35548,36499,...,331425,339187,334290,330437,331233,329936,333699,334078,335987,363454
1,AL,508500,473290,497504,505064,558585,614582,642039,681691,720272,...,804136,835603,850483,861148,823667,813733,827644,845914,819961,773188
2,AR,204729,199953,216451,230995,255283,255234,257650,277382,285240,...,421863,405171,412224,412792,377648,381146,397376,411175,400559,377372
3,AZ,50359,81565,97937,105519,100297,119098,131975,119078,128483,...,224630,219917,213684,236623,236019,240833,231833,229212,226423,224452
4,CA,1193589,1209024,1214818,1325160,1454644,1475421,1558212,1610012,1697912,...,1783641,1746357,1884236,1859708,1849423,1851514,1828150,1852054,1806508,1701239


# Transformation of Data

In [19]:
df_trans = df.melt(id_vars=['State'] , var_name='Year', value_name='Yearly Data')
df_trans['Year'] = pd.to_datetime(df_trans['Year'], format='%Y')

df_trans


,State,Year,Yearly Data
0,AK,1960-01-01,16798
1,AL,1960-01-01,508500
2,AR,1960-01-01,204729
3,AZ,1960-01-01,50359
4,CA,1960-01-01,1193589
...,...,...,...
3167,WA,2020-01-01,479089
3168,WI,2020-01-01,537916
3169,WV,2020-01-01,380646
3170,WY,2020-01-01,291971


# Modeling the Data
## Using ARIMA Model

In [20]:
os.makedirs('Plots/use_tot_Sector_data_Industrial_Sector/Arima_results_plots',exist_ok=True)

for State in df_trans['State'].unique():
        try:
            
            fig = go.Figure()

            # Get the energy consumption data for the current country and sector
            df_filter = df_trans[df_trans['State'] == State ][['Year', 'Yearly Data']]
            df_filter_index = df_filter.set_index('Year')

            train_data = df_filter[:-5]
            test_data = df_filter[-5:]
            
            # Prepare the data for modeling
            years = df_filter_index.index
            energy_consumption = df_filter_index.values.flatten()

                    # Split the data into training and testing
            # Use all data except the last 5 years for training
            Horizan = -5
            train_data = energy_consumption[:Horizan]
            test_data = energy_consumption[Horizan:]  # Use the last 5 years for testing

            # Fit the auto ARIMA model
            model = auto_arima(train_data, seasonal=False)
            model.fit(train_data)

            # Generate predictions
            predictions = model.predict(n_periods=len(test_data))
            predictions_ahead_in_future = model.predict(n_periods=len(test_data)+15)

            # Calculate evaluation metrics
            mae = mean_absolute_error(test_data, predictions)
            mse = mean_squared_error(test_data, predictions)
            mape = np.mean(np.abs((test_data - predictions) / test_data)) * 100

            print('Mean Absolute Error (MAE):', np.round(mae,2))
            print('Mean Squared Error (MSE):', np.round(mse,2))
            print('Mean Absolute Percentage Error (MAPE):', np.round(mape,2))
            
            # Plot the training data
            fig.add_trace(go.Scatter(
                x=years[:Horizan], y=train_data, mode='lines+markers', name='Training Data'))

            # Plot the predictions
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=test_data, mode='lines+markers', name='Actual'))
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=predictions, mode='lines+markers', name='Predicted'))

            fig.add_trace(go.Scatter(
                x=pd.date_range(start = years[Horizan],periods=15,freq='Y'), y=predictions_ahead_in_future, mode='lines+markers', name='Prediction till 2030'))

            # Update the layout
            fig.update_layout(title=f'Industrial Sector : Energy Consumption Forecast State : {State} using ARIMA Model',
                            xaxis_title='Year', yaxis_title='Energy Consumption')

            # Show the plot
            fig.show()
            # print(State,msn)
            fig.write_image(f'Plots/use_tot_Sector_data_Industrial_Sector/Arima_results_plots/{State}.png')
            # break
        except:
            print('Error occoured in Combination State : {} and MSN : {} Due NaN Value'.format(State,mse))
        

Mean Absolute Error (MAE): 10407.6
Mean Squared Error (MSE): 135124069.6
Mean Absolute Percentage Error (MAPE): 3.1


Mean Absolute Error (MAE): 18068.6
Mean Squared Error (MSE): 634258754.2
Mean Absolute Percentage Error (MAPE): 2.26


Mean Absolute Error (MAE): 15988.0
Mean Squared Error (MSE): 410095962.8
Mean Absolute Percentage Error (MAPE): 3.97


Mean Absolute Error (MAE): 16170.65
Mean Squared Error (MSE): 350224254.51
Mean Absolute Percentage Error (MAPE): 7.12


Mean Absolute Error (MAE): 43418.8
Mean Squared Error (MSE): 4852806010.4
Mean Absolute Percentage Error (MAPE): 2.5


...

Mean Absolute Error (MAE): 47015.64
Mean Squared Error (MSE): 3276897595.88
Mean Absolute Percentage Error (MAPE): 9.22


Mean Absolute Error (MAE): 25557.44
Mean Squared Error (MSE): 1420930075.28
Mean Absolute Percentage Error (MAPE): 4.58


Mean Absolute Error (MAE): 34603.2
Mean Squared Error (MSE): 1688639776.4
Mean Absolute Percentage Error (MAPE): 9.41


Mean Absolute Error (MAE): 18114.49
Mean Squared Error (MSE): 416947240.66
Mean Absolute Percentage Error (MAPE): 5.75


Mean Absolute Error (MAE): 666666.6
Mean Squared Error (MSE): 768490782569.4
Mean Absolute Percentage Error (MAPE): 2.05


## Using SARIMA Model

In [21]:
os.makedirs('Plots/use_tot_Sector_data_Industrial_Sector/Sarima_results_plots',exist_ok=True)

for State in df_trans['State'].unique():
        try:
            
            fig = go.Figure()

            # Get the energy consumption data for the current country and sector
            df_filter = df_trans[df_trans['State'] == State ][['Year', 'Yearly Data']]
            df_filter_index = df_filter.set_index('Year')

            train_data = df_filter[:-5]
            test_data = df_filter[-5:]
            
            # Prepare the data for modeling
            years = df_filter_index.index
            energy_consumption = df_filter_index.values.flatten()

                    # Split the data into training and testing
            # Use all data except the last 5 years for training
            Horizan = -5
            train_data = energy_consumption[:Horizan]
            test_data = energy_consumption[Horizan:]  # Use the last 5 years for testing

            # Fit the auto ARIMA model
            model = auto_arima(train_data, seasonal=False)
            model.fit(train_data)

            # Generate predictions
            predictions = model.predict(n_periods=len(test_data))
            predictions_ahead_in_future = model.predict(n_periods=len(test_data)+15)

            # Calculate evaluation metrics
            mae = mean_absolute_error(test_data, predictions)
            mse = mean_squared_error(test_data, predictions)
            mape = np.mean(np.abs((test_data - predictions) / test_data)) * 100

            print('Mean Absolute Error (MAE):', np.round(mae,2))
            print('Mean Squared Error (MSE):', np.round(mse,2))
            print('Mean Absolute Percentage Error (MAPE):', np.round(mape,2))
            
            # Plot the training data
            fig.add_trace(go.Scatter(
                x=years[:Horizan], y=train_data, mode='lines+markers', name='Training Data'))

            # Plot the predictions
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=test_data, mode='lines+markers', name='Actual'))
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=predictions, mode='lines+markers', name='Predicted'))

            fig.add_trace(go.Scatter(
                x=pd.date_range(start = years[Horizan],periods=15,freq='Y'), y=predictions_ahead_in_future, mode='lines+markers', name='Prediction till 2030'))

            # Update the layout
            fig.update_layout(title=f'Industrial Sector : Energy Consumption Forecast State : {State} using SARIMA Model',
                            xaxis_title='Year', yaxis_title='Energy Consumption')

            # Show the plot
            fig.show()
            # print(State,msn)
            fig.write_image(f'Plots/use_tot_Sector_data_Industrial_Sector/Sarima_results_plots/{State}.png')
            # break
        except:
            print('Error occoured in Combination State : {} and MSN : {} Due NaN Value'.format(State,mse))
        # break

Mean Absolute Error (MAE): 10407.6
Mean Squared Error (MSE): 135124069.6
Mean Absolute Percentage Error (MAPE): 3.1


Mean Absolute Error (MAE): 18068.6
Mean Squared Error (MSE): 634258754.2
Mean Absolute Percentage Error (MAPE): 2.26


Mean Absolute Error (MAE): 15988.0
Mean Squared Error (MSE): 410095962.8
Mean Absolute Percentage Error (MAPE): 3.97


Mean Absolute Error (MAE): 16170.65
Mean Squared Error (MSE): 350224254.51
Mean Absolute Percentage Error (MAPE): 7.12


Mean Absolute Error (MAE): 43418.8
Mean Squared Error (MSE): 4852806010.4
Mean Absolute Percentage Error (MAPE): 2.5


...

Mean Absolute Error (MAE): 47015.64
Mean Squared Error (MSE): 3276897595.88
Mean Absolute Percentage Error (MAPE): 9.22


Mean Absolute Error (MAE): 25557.44
Mean Squared Error (MSE): 1420930075.28
Mean Absolute Percentage Error (MAPE): 4.58


Mean Absolute Error (MAE): 34603.2
Mean Squared Error (MSE): 1688639776.4
Mean Absolute Percentage Error (MAPE): 9.41


Mean Absolute Error (MAE): 18114.49
Mean Squared Error (MSE): 416947240.66
Mean Absolute Percentage Error (MAPE): 5.75


Mean Absolute Error (MAE): 666666.6
Mean Squared Error (MSE): 768490782569.4
Mean Absolute Percentage Error (MAPE): 2.05
